In [1]:
import telebot #pip install pyTelegramBotAPI
from datetime import datetime, timezone, timedelta
import mysql.connector #pip install mysql.connector
import pytz
cordoba_tz = pytz.timezone('America/Argentina/Cordoba') # Definir zona horaria

In [2]:
# Conectarse a la base de datos
def load_message_db(id_mensaje, id_chat, texto, usuario, fecha_hora):
    # crear conector
    mydb = mysql.connector.connect(
        host="db-interop-gestion-instance-1.crssqp6kq2v3.us-east-1.rds.amazonaws.com",
        user="admin",
        password="EPXf2NVXzYi7",
        database="telegram"
    )
    # Crear un cursor para ejecutar consultas
    cursor = mydb.cursor()
    # Hacer insert
    sql = '''INSERT INTO mensajes_texto 
    (id_mensaje, id_chat, texto, usuario, fecha_hora) 
    VALUES (%s, %s, %s, %s, %s)'''
    # Tupla de valores para la consulta
    values = (id_mensaje, id_chat, texto, usuario, fecha_hora)
    # Ejecutar la consulta con los valores como parámetros
    cursor.execute(sql, values)
    mydb.commit()
    # Cerrar conexión
    cursor.close()
    mydb.close()

In [3]:
# Crea la instancia del bot
bot_token = '6794338457:AAEKvwMr7Ql6PrDVBYrhMHFoWIKRHtHzJaU'
bot = telebot.TeleBot(bot_token)

In [4]:
@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.reply_to(message, "¡Hola! Bienvenido al bot de pruebas.")

In [5]:
# Manejador de mensajes
@bot.message_handler(func=lambda message: True)
def handle_messages(message):
    # Extraer id, grupo, texto, usuario y fecha
    message_id = message.message_id
    chat_id = message.chat.id 
    text = message.text
    fecha_hora_actual = datetime.now(cordoba_tz)
    date = fecha_hora_actual.strftime('%Y-%m-%d %H:%M:%S')
    user = message.from_user
    username = user.username
    # Verificar si el nombre de usuario está presente
    if username:
        user_info = username
    else:
        first_name = user.first_name
        last_name = user.last_name if user.last_name else ""
        user_info = f"{first_name}{last_name}" if (first_name or last_name) else "Sin Nombre"    
    load_message_db(message_id, chat_id, text, user_info, date)

In [ ]:
# Inicia el bot
bot.polling(none_stop=True)